In [1]:
# Load IPFX libraries
from ipfx.dataset.create import create_ephys_data_set
from ipfx.data_set_features import extract_data_set_features
from ipfx.utilities import drop_failed_sweeps
from ipfx.qc_feature_extractor import sweep_qc_features
from ipfx.stimulus import StimulusOntology
from ipfx.data_set_features import extract_cell_long_square_features

from ipfx.feature_extractor import SpikeFeatureExtractor, SpikeTrainFeatureExtractor
import ipfx.stimulus_protocol_analysis as spa
from ipfx.epochs import get_stim_epoch

#Load Plotting 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# these are basic IPFX objects for working with sweeps and sets of sweeps
from ipfx.sweep import Sweep, SweepSet 

# Load pandas library too
import pandas as pd
from ipfx.dataset.create import create_ephys_data_set
from ipfx.utilities import drop_failed_sweeps

#Load json
import json
import allensdk.core.json_utilities as ju

In [2]:
def summarize_ls_ephys_features(lsa_results):
    hero_keep_features = ['adapt', 'avg_rate', 'first_isi', 'isi_cv', 'latency', 'mean_isi', 'median_isi', 
                          'stim_amp']
    rheo_keep_features = ['threshold_v', 'peak_v', 'trough_v', 
     'fast_trough_v', 'adp_v', 'width', 'upstroke_downstroke_ratio', 'peak_t', 'fast_trough_t', 'trough_t']
    
    overall_cell_keep_features = ['v_baseline', 'rheobase_i', 'fi_fit_slope', 
                                  'sag', 'vm_for_sag', 'input_resistance', 'tau']
    
    hero_small_dict = lsa_results['hero_sweep'][hero_keep_features]
    rheobase_sweep_index = lsa_results['rheobase_sweep'].name
    rheobase_sweep = lsa_results['spikes_set'][rheobase_sweep_index].iloc[0]
    
    rheo_spike_small_dict = rheobase_sweep[rheo_keep_features]
    rheo_first_isi = lsa_results['rheobase_sweep']['first_isi']
    rheo_spike_small_dict['rheo_first_isi'] = rheo_first_isi

    spike_comb_dict = {**hero_small_dict, **rheo_spike_small_dict}

    overall_cell_features = {x: lsa_results[x] for x in overall_cell_keep_features if x in lsa_results}
    final_cell_feature_dict = {**spike_comb_dict, **overall_cell_features}
    return(final_cell_feature_dict)

In [3]:
from pathlib import Path
import utils

In [4]:
h2019 = utils.summary_df(Path('./data/processed/recordings/human2019/'), suffix='.nwb')
h2018 = utils.summary_df(Path('./data/processed/recordings/human2018/'), suffix='.nwb')
m2019 = utils.summary_df(Path('./data/processed/recordings/mouse2019/'), suffix='.nwb')
m2018 = utils.summary_df(Path('./data/processed/recordings/mouse2018/'), suffix='.nwb')

# add file_id to merge with metadata
h2019['file_id'] = h2019.file_name.apply(lambda x: x.rstrip('.nwb'))
h2018['file_id'] = h2018.file_name.apply(lambda x: x.rstrip('.nwb'))
m2019['file_id'] = m2019.file_name.apply(lambda x: x.rstrip('.nwb'))
m2018['file_id'] = m2018.file_name.apply(lambda x: x.rstrip('.nwb'))

# meta 
h2019meta = pd.read_csv('./data/raw/Metadata/Human19_Metadatav2.csv', index_col=0)
h2019meta = h2019meta[~h2019meta.stimulus_type.isna()]
h2018meta = pd.read_csv('./data/raw/Metadata/Human18_Metadatav2.csv', index_col=0)
h2018meta = h2018meta[~h2018meta['Stimulus type'].isna()]
m2019meta = pd.read_csv('./data/raw/Metadata/Mouse19_Metadatav2.csv', index_col=0)
m2019meta = m2019meta[~m2019meta.stimulus_type.isna()]
m2018meta = pd.read_csv('./data/raw/Metadata/Mouse18_Metadatav2.csv', index_col=0)
m2018meta = m2018meta[~m2018meta.stimulus_type.isna()]

In [5]:
features_output_path = Path('./data/processed/features')
features_output_path.mkdir(exist_ok=True)

## Process 2019 human longsquares

In [6]:
h2019meta[h2019meta.stimulus_type == 'longsquare'].head()

,ABF File,Data Type,Exp. Date,Cell #,Cell Layer,Stim Type,Threshold,DC,ZD,Hold,Gain,Offset,Response Channel,Command Channel,RMP,stimulus_type
217,2019_11_04_0098,Human,Nov_04_2019,C2,L3C,Intrinsic Properties,n.a,n.a,ZD,n.a,1,0,Vm_Sec(mV),IN 12(pA),-60.0,longsquare
229,2019_11_04_0128,Human,Nov_04_2019,C3,L5,Intrinsic Properties,n.a,n.a,ZD,n.a,1,0,Vm_Sec(mV),IN 12(pA),-60.0,longsquare
260,2019_11_26_0103,Human,Nov_26_2019,C8,L5,Intrinsic Properties,n.a,n.a,ZD,n.a,1,0,Vm_Sec(mV),IN 12(pA),-65.2,longsquare
264,2019_11_28_0016,Human,Nov_28_2019,C1,L3C,Intrinsic Properties,n.a,n.a,ZD,n.a,1,0,Vm_Sec(mV),IN 12(pA),-62.4,longsquare
271,2019_11_28_0054,Human,Nov_28_2019,C2,L3C,Intrinsic Properties,n.a,n.a,ZD,n.a,1,0,Vm_Sec(mV),IN 12(pA),-70.0,longsquare


In [7]:
cols_to_drop = ['ABF File', 'Data Type', 'Exp. Date', 'Threshold', 'DC', 'ZD', 'Hold']
h2019 = h2019.merge(h2019meta, left_on='file_id', right_on='ABF File').drop(cols_to_drop, axis=1)

In [8]:
h2019[h2019.stimulus_type == 'longsquare'].shape

(56, 12)

In [9]:
ontology_path = "./data/stim_ontology_new_Homeira.json"

In [10]:
from ipfx.error import FeatureError

In [11]:
data_dict = {}
for i, row in h2019[h2019.stimulus_type == 'longsquare'].iterrows():
    print(f'{row.file_name}')
    try:
        data_set = create_ephys_data_set(nwb_file=row.path, ontology=ontology_path)
        cell_features, sweep_features, cell_record, sweep_records, _, _ = extract_data_set_features(data_set)
        long_square_table = data_set.filtered_sweep_table(stimuli=data_set.ontology.long_square_names)
        long_square_sweeps = data_set.sweep_set(long_square_table.sweep_number)
        long_square_sweeps = data_set.sweep_set()

        # Select epoch corresponding to the actual recording from the sweeps
        # and align sweeps so that the experiment would start at the same time
        long_square_sweeps.select_epoch("recording")

        # find the start and end time of the stimulus
        # (treating the first sweep as representative)
        stim_start_index, stim_end_index = get_stim_epoch(long_square_sweeps.i[0], test_pulse=False)
        stim_start_time = long_square_sweeps.t[0][stim_start_index]
        stim_end_time = long_square_sweeps.t[0][stim_end_index]

        stim_start_time = 0.2
        stim_end_time = 1.8

        # build the extractors
        spfx = SpikeFeatureExtractor(start=stim_start_time, end=stim_end_time, filter = 1)
        sptfx = SpikeTrainFeatureExtractor(start=stim_start_time, end=stim_end_time)

        # run the analysis and print out a few of the features
        long_square_analysis = spa.LongSquareAnalysis(spfx, sptfx, subthresh_min_amp=-100.0)
        data = long_square_analysis.analyze(long_square_sweeps)    

        specimen_dict = summarize_ls_ephys_features(data)
        #data_dict[curr_file] = specimen_dict
        data_dict[row.file_name] = specimen_dict
        
    except (AssertionError, AttributeError, TypeError, FeatureError) as e:
    #except FeatureError as e:
        print(f'Error: {e} \n') # {row.filename} \n')

19320017.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
INFO:numexpr.utils:NumExpr defaulting to 8 t

19320001.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephys

19228058.nwb


...

INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
INFO:root:Assigned subthreshold minimum amplitude of -100.000000.
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus_protocol_analysis.py:219: RuntimeWarning: Mean of empty slice
  features["tau"]

19o10010.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
INFO:root:Assigned subthreshold minimum amplitude of -100.000000.
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus_protocol_analysis.py:219: RuntimeWarning: Mean of empty slice
  features["tau"]

2019_11_04_0001.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
INFO:root:Assigned subthreshold minimum amplitude of -100.000000.
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus_protocol_analysis.py:219: RuntimeWarning: Mean of empty slice
  features["tau"]

In [12]:
dt_df = pd.DataFrame(data_dict)
h2019_longsquare_df = dt_df.T

In [13]:
h2019_longsquare_df

,adapt,avg_rate,first_isi,isi_cv,latency,mean_isi,median_isi,stim_amp,threshold_v,peak_v,...,fast_trough_t,trough_t,rheo_first_isi,v_baseline,rheobase_i,fi_fit_slope,sag,vm_for_sag,input_resistance,tau
19320017.nwb,0.232653,2.500,0.06180,0.372042,0.04440,0.107767,0.102000,150.000000,-31.875294,27.731640,...,0.56650,0.94710,NaN,-63.793605,100.000000,0.020089,0.823586,-63.479305,128.177795,NaN
19320001.nwb,0.307538,3.125,0.01890,0.531679,0.03350,0.100975,0.115450,150.000000,-35.537796,38.839649,...,0.29880,0.29900,0.03050,-55.632836,100.000000,0.022321,0.967081,-55.239941,112.905121,NaN
19228058.nwb,NaN,1.250,0.02090,0.000000,0.04990,0.020900,0.020900,100.000000,-37.721977,21.579784,...,0.49610,0.49720,0.02600,-55.205246,50.000000,0.012054,1.672227,-55.336720,137.025070,NaN
19129072.nwb,0.107086,6.250,0.00580,0.332472,0.00150,0.063522,0.068600,100.000000,-37.769829,20.933790,...,0.26080,0.26070,0.00800,-58.896030,50.000000,0.025000,-0.494377,-56.458794,194.061737,NaN
19129058.nwb,0.095315,4.375,0.04340,0.289976,0.01660,0.088317,0.096400,150.000000,-35.190723,16.933302,...,0.28340,0.30480,0.13840,-57.294895,100.000000,0.006429,0.769879,-57.554008,124.398575,NaN
19320007.nwb,0.178172,2.500,0.09990,0.283699,0.02120,0.151867,0.150300,200.000000,-38.216209,34.452248,...,0.27100,0.30930,0.30880,-56.197939,150.000000,0.014286,1.548829,-55.696682,70.997772,NaN
19328039.nwb,0.115475,7.500,0.00880,0.543804,0.00260,0.049991,0.061300,100.000000,-34.691113,36.378223,...,0.23850,0.23840,0.00790,-52.917082,50.000000,0.011654,1.015691,-51.646679,197.092285,NaN
19319025.nwb,0.139882,4.375,0.03430,0.408718,0.00550,0.083083,0.082100,150.000000,-43.015823,23.658987,...,0.23810,0.23890,0.07900,-58.946423,100.000000,0.021071,-2.962848,-58.689651,142.304306,NaN
19128040.nwb,0.169593,2.500,0.04730,0.266636,0.03450,0.071733,0.073900,150.000000,-42.352150,26.837307,...,0.21980,0.21980,0.06100,-57.215582,100.000000,-0.002500,1.598834,-56.951759,153.752518,NaN
19128068.nwb,0.590410,1.875,0.08200,0.590410,0.05040,0.200200,0.200200,249.999985,-35.180370,38.598930,...,0.32590,0.92980,NaN,-53.913972,200.000000,0.018750,0.123652,-53.576370,49.600067,NaN


In [14]:
h2019_longsquare_df.to_csv(features_output_path / 'human2019_longsquares.csv')

In [15]:
h2019_longsquare_df.index

Index(['19320017.nwb', '19320001.nwb', '19228058.nwb', '19129072.nwb',
       '19129058.nwb', '19320007.nwb', '19328039.nwb', '19319025.nwb',
       '19128040.nwb', '19128068.nwb', '19129014.nwb', '19122026.nwb',
       '19328001.nwb', '19129037.nwb', '19228030.nwb', '19122003.nwb',
       '19328019.nwb', '19320041.nwb', '19129022.nwb', '19128061.nwb',
       '19129024.nwb', '19328009.nwb', '19328034.nwb', '19129040.nwb',
       '19328046.nwb', '19320022.nwb', '19129043.nwb', '19228044.nwb',
       '19320025.nwb', '19128006.nwb', '19228068.nwb', '19320030.nwb',
       '19129047.nwb', '2019_11_28_0010.nwb', '2019_11_28_0038.nwb',
       '2019_11_26_0019.nwb', '2019_11_28_0016.nwb', '2019_11_26_0037.nwb',
       '19o10045.nwb', '2019_11_04_0083.nwb', '2019_11_26_0094.nwb',
       '19o10054.nwb', '2019_11_26_0110.nwb', '2019_11_28_0079.nwb',
       '2019_11_28_0119.nwb', '2019_11_04_0128.nwb', '19o10065.nwb',
       '2019_11_28_0127.nwb', '2019_11_28_0090.nwb', '2019_11_26_0103.nwb',
    

In [16]:
h2019_LS_error = h2019[(h2019.stimulus_type == 'longsquare') & ~(h2019.file_name.isin(h2019_longsquare_df.index))]

In [17]:
h2019_LS_error.path.values[0]

'data/processed/recordings/human2019/abfv2/2019_11_28_0114.nwb'

### Notes for error with  'data/processed/recordings/human2019/abfv2/2019_11_28_0114.nwb'
```
2019_11_28_0114.nwb
INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
INFO:root:Assigned subthreshold minimum amplitude of -100.000000.
WARNING:root:cannot unpack non-iterable NoneType object
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
WARNING:root:Cannot find ('Short Square', 'Short Square Threshold', 'Short Square - Hold -60mV', 'Short Square - Hold -70mV', 'Short Square - Hold -80mV') sweeps with clamp mode: CurrentClamp 
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
WARNING:root:Cannot find ('Ramp',) sweeps with clamp mode: CurrentClamp 
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
WARNING:root:cannot unpack non-iterable NoneType object
Error: No spiking long square sweeps, cannot compute cell features. 
```

In [18]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

In [19]:
io = NWBHDF5IO(h2019_LS_error.path.values[0], mode='r')
nwb = io.read()

nwb2widget(nwb)

In [20]:
io.close()

# Process 2018 human longsquares

In [21]:
h2018meta['Stimulus type'].value_counts()

Long_Square    22
Ramp           20
Name: Stimulus type, dtype: int64

In [22]:
h2018meta.columns

Index(['ABF File', 'Data Type', 'Exp. Date', 'Case #', 'Cell #', 'Cell Layer',
       'Stim Type', 'Threshold', 'Gain Value', 'ZD', 'Hold', 'Unnamed: 12',
       'Gain', 'Offset', 'Response Channel', 'Command Channel', 'RMP',
       'Stimulus type'],
      dtype='object')

In [23]:
h2018meta[h2018meta['Stimulus type'] == 'Long_Square'].head()

,ABF File,Data Type,Exp. Date,Case #,Cell #,Cell Layer,Stim Type,Threshold,Gain Value,ZD,Hold,Unnamed: 12,Gain,Offset,Response Channel,Command Channel,RMP,Stimulus type
9,18320005,Human,March_20_2018,First,C1,L23,Firing,n.a,n.a,n.a,n.a,NaN,1.0,-20.0,Iclamp(mV),Current_in(pA),-70.5,Long_Square
13,18320015,Human,March_20_2018,First,C2,L23,Firing,n.a,n.a,n.a,n.a,NaN,1.0,-20.0,Iclamp(mV),Current_in(pA),-68.1,Long_Square
19,18320021,Human,March_20_2018,First,C3,L23,Firing,n.a,n.a,n.a,n.a,NaN,1.0,-27.0,Iclamp(mV),Current_in(pA),-71.3,Long_Square
30,18320014,Human,March_20_2018,Second,C2,L23,Firing,n.a,n.a,n.a,n.a,NaN,1.0,-17.2,Iclamp(mV),Current_in(pA),-65.2,Long_Square
33,18320019,Human,March_20_2018,Second,C3,L23,Firing,n.a,n.a,n.a,n.a,NaN,1.0,-18.2,Iclamp(mV),Current_in(pA),-66.2,Long_Square


In [24]:
cols_to_drop_h2018 = ['ABF File', 'Data Type', 'Exp. Date', 'Stim Type', 'Threshold', 'Gain Value', 'ZD', 'Hold', 'Unnamed: 12', 'Response Channel', 'Command Channel']
h2018 = h2018.merge(h2018meta, left_on='file_id', right_on='ABF File').drop(cols_to_drop_h2018, axis=1)

In [25]:
h2018.head()

,file_name,path,file_id,Case #,Cell #,Cell Layer,Gain,Offset,RMP,Stimulus type
0,18329051.nwb,data/processed/recordings/human2018/18329051.nwb,18329051,n.a,C10,L23,1.0,-24.7,-69.7,Long_Square
1,18329044.nwb,data/processed/recordings/human2018/18329044.nwb,18329044,n.a,C9.2,L23-Int,1.0,-19.0,-69.7,Long_Square
2,18329050.nwb,data/processed/recordings/human2018/18329050.nwb,18329050,n.a,C10,L23,1.0,-24.7,-69.7,Ramp
3,18o22020.nwb,data/processed/recordings/human2018/18o22020.nwb,18o22020,n.a,C3,L23,20.0,-20.0,-68.4,Long_Square
4,18129004.nwb,data/processed/recordings/human2018/18129004.nwb,18129004,n.a,C2,L5,1.0,-17.2,-71.2,Long_Square


In [26]:
data_dict = {}
for i, row in h2018[h2018['Stimulus type'] == 'Long_Square'].iterrows():
    print(f'{row.file_name}')
    try:
        data_set = create_ephys_data_set(nwb_file=row.path, ontology=ontology_path)
        cell_features, sweep_features, cell_record, sweep_records, _, _ = extract_data_set_features(data_set)
        long_square_table = data_set.filtered_sweep_table(stimuli=data_set.ontology.long_square_names)
        long_square_sweeps = data_set.sweep_set(long_square_table.sweep_number)
        long_square_sweeps = data_set.sweep_set()

        # Select epoch corresponding to the actual recording from the sweeps
        # and align sweeps so that the experiment would start at the same time
        long_square_sweeps.select_epoch("recording")

        # find the start and end time of the stimulus
        # (treating the first sweep as representative)
        stim_start_index, stim_end_index = get_stim_epoch(long_square_sweeps.i[0], test_pulse=False)
        stim_start_time = long_square_sweeps.t[0][stim_start_index]
        stim_end_time = long_square_sweeps.t[0][stim_end_index]

        stim_start_time = 0.2
        stim_end_time = 1.8

        # build the extractors
        spfx = SpikeFeatureExtractor(start=stim_start_time, end=stim_end_time, filter = 1)
        sptfx = SpikeTrainFeatureExtractor(start=stim_start_time, end=stim_end_time)

        # run the analysis and print out a few of the features
        long_square_analysis = spa.LongSquareAnalysis(spfx, sptfx, subthresh_min_amp=-100.0)
        data = long_square_analysis.analyze(long_square_sweeps)    

        specimen_dict = summarize_ls_ephys_features(data)
        #data_dict[curr_file] = specimen_dict
        data_dict[row.file_name] = specimen_dict
        
    except (AssertionError, AttributeError, TypeError, FeatureError) as e:
    #except FeatureError as e:
        print(f'Error: {e} \n') # {row.filename} \n')

18329051.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
INFO:root:Need to recalculate threshold-peak

18329044.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
INFO:root:Need to recalculate threshold-peak

18o22020.nwb


...

INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephys

18329062.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  


Error: No spiking long square sweeps, cannot compute cell features. 

18o22010.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephys

In [27]:
dt_df = pd.DataFrame(data_dict)
h2018_longsquare_df = dt_df.T

In [28]:
h2018[h2018['Stimulus type'] == 'Long_Square'].shape

(21, 10)

In [29]:
h2018_longsquare_df

,adapt,avg_rate,first_isi,isi_cv,latency,mean_isi,median_isi,stim_amp,threshold_v,peak_v,...,fast_trough_t,trough_t,rheo_first_isi,v_baseline,rheobase_i,fi_fit_slope,sag,vm_for_sag,input_resistance,tau
18329051.nwb,0.075939,4.375,0.0937,0.272518,0.0196,0.130567,0.12110,160.000000,-40.263966,34.198925,...,0.2407,0.2549,0.1603,-64.277105,119.999992,0.002790,118.507759,-101.604301,152.587891,NaN
18329044.nwb,NaN,1.250,0.2489,0.000000,0.0675,0.248900,0.24890,80.000000,-64.471191,11.212403,...,0.9506,1.6601,NaN,-67.548129,40.000000,0.019799,53.749706,-99.566406,213.623047,NaN
18o22020.nwb,-0.015676,3.750,0.1467,0.056414,0.1105,0.137120,0.13530,160.000000,-8168.193848,23081.808594,...,0.3627,0.3981,0.1697,-16755.909570,119.999992,0.052232,0.138446,-10328.837891,100250.244141,NaN
18129004.nwb,0.026685,7.500,0.0512,0.190403,0.0142,0.071336,0.06960,100.000000,-42.834770,19.421093,...,0.2143,0.2198,0.0511,-60.945889,59.999996,0.005787,-187.527481,-89.831833,267.028838,NaN
18320014.nwb,0.039294,4.375,0.0954,0.119876,0.0712,0.118667,0.12065,180.000000,-32.763966,54.211136,...,0.3632,0.3772,0.2908,-60.975262,140.000000,0.031250,362.698364,-91.357712,122.070327,NaN
18426010.nwb,0.026073,5.000,0.0803,0.111064,-0.0009,0.100062,0.10050,80.000000,-27.812206,39.936817,...,0.2438,0.2543,0.0994,-68.486061,40.000000,0.010937,-277.900818,-103.800980,251.770020,NaN
18320005.nwb,0.042092,3.750,0.1210,0.128160,0.0653,0.148200,0.15010,100.000000,-45.634769,24.555664,...,0.3901,1.3123,NaN,-62.719860,59.999996,0.039621,-99.743736,-99.956055,205.993667,NaN
18426017.nwb,0.029396,5.625,0.0737,0.145713,0.0417,0.090412,0.08855,200.000000,-32.810551,52.638676,...,0.2448,0.2465,0.0846,-56.585329,160.000000,0.015625,193.321381,-96.592293,137.329102,NaN
18201011.nwb,0.042872,5.625,0.0630,0.172232,0.0183,0.091400,0.09360,90.000000,-44.881351,44.535160,...,0.2218,0.2587,0.0950,-50.973503,50.000000,0.052455,55.410137,-84.249023,312.805176,NaN
18220008.nwb,0.009467,8.750,0.0517,0.086322,0.0458,0.059777,0.06180,80.000000,-46.914062,43.112797,...,0.2842,0.2964,0.1193,-92.034408,29.999998,0.031016,231.485641,-93.300781,451.660156,NaN


In [30]:
h2018_longsquare_df.to_csv(features_output_path / 'human2018_longsquares.csv')

In [31]:
h2018_LS_error = h2018[(h2018['Stimulus type'] == 'Long_Square') & ~(h2018.file_name.isin(h2018_longsquare_df.index))]

In [32]:
h2018_LS_error

,file_name,path,file_id,Case #,Cell #,Cell Layer,Gain,Offset,RMP,Stimulus type
38,18329062.nwb,data/processed/recordings/human2018/18329062.nwb,18329062,n.a,C13,L23,1.0,-16.2,-61.2,Long_Square


### Notes for error with data/processed/recordings/human2018/18329062.nwb
Looking at feature extraction steps copied from above:
```
18329062.nwb
INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning:

Could not find stimulus: NA

WARNING:root:Cannot find ('Long Square', 'Long Square Threshold', 'Long Square SupraThreshold', 'Long Square SubThreshold') sweeps with clamp mode: CurrentClamp
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
WARNING:root:Cannot find ('Short Square', 'Short Square Threshold', 'Short Square - Hold -60mV', 'Short Square - Hold -70mV', 'Short Square - Hold -80mV') sweeps with clamp mode: CurrentClamp 
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
WARNING:root:Cannot find ('Ramp',) sweeps with clamp mode: CurrentClamp 
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
WARNING:root:cannot unpack non-iterable NoneType object
Error: No spiking long square sweeps, cannot compute cell features.
```

In [33]:
io = NWBHDF5IO(h2018_LS_error.path.values[0], mode='r')
nwb = io.read()

nwb2widget(nwb)

In [34]:
io.close()

## Process 2019 mouse longsquares

In [35]:
m2019meta.stimulus_type.value_counts()

ramp           20
longsquare     16
shortsquare    14
Name: stimulus_type, dtype: int64

In [36]:
m2019meta.head()

,Cell Name,Data Type,Exp. Date,Cell #,Cell Layer,Stim Type,ZD status,stimulus_type,Gain,Offset,Response Channel,Command Channel,RMP
4,19o22017,Mouse,Oct 22_2019,C2,L5,Firing,After ZD,shortsquare,1.0,0.0,Vm_Sec (mV),IN 11(pA),-66.0
5,19o22001,Mouse,Oct 22_2019,C2,L5,Firing,Before ZD,longsquare,1.0,0.0,Vm_Sec (mV),IN 11(pA),-64.0
19,19o22031,Mouse,Oct 22_2019,C3,L5,Firing,After ZD,longsquare,1.0,0.0,Vm_Sec (mV),IN 11(pA),-70.0
23,19o22037,Mouse,Oct 22_2019,C3,L5,Firing,After ZD,ramp,1.0,0.0,Vm_Sec (mV),IN 11(pA),-67.0
24,19o22018,Mouse,Oct 22_2019,C3,L5,Firing,Before ZD,longsquare,1.0,0.0,Vm_Sec (mV),IN 11(pA),-75.0


In [37]:
cols_to_drop_m2019 = ['Cell Name', 'Data Type', 'Exp. Date', 'Stim Type', 'ZD status', 'Response Channel', 'Command Channel']
m2019 = m2019.merge(m2019meta, left_on='file_id', right_on='Cell Name').drop(cols_to_drop_m2019, axis=1)

In [38]:
m2019[m2019.stimulus_type == 'longsquare'].shape

(16, 9)

In [39]:
data_dict = {}
for i, row in m2019[m2019.stimulus_type == 'longsquare'].iterrows():
    print(f'{row.file_name}')
    try:
        data_set = create_ephys_data_set(nwb_file=row.path, ontology=ontology_path)
        cell_features, sweep_features, cell_record, sweep_records, _, _ = extract_data_set_features(data_set)
        long_square_table = data_set.filtered_sweep_table(stimuli=data_set.ontology.long_square_names)
        long_square_sweeps = data_set.sweep_set(long_square_table.sweep_number)
        long_square_sweeps = data_set.sweep_set()

        # Select epoch corresponding to the actual recording from the sweeps
        # and align sweeps so that the experiment would start at the same time
        long_square_sweeps.select_epoch("recording")

        # find the start and end time of the stimulus
        # (treating the first sweep as representative)
        stim_start_index, stim_end_index = get_stim_epoch(long_square_sweeps.i[0], test_pulse=False)
        stim_start_time = long_square_sweeps.t[0][stim_start_index]
        stim_end_time = long_square_sweeps.t[0][stim_end_index]

        stim_start_time = 0.2
        stim_end_time = 1.8

        # build the extractors
        spfx = SpikeFeatureExtractor(start=stim_start_time, end=stim_end_time, filter = 1)
        sptfx = SpikeTrainFeatureExtractor(start=stim_start_time, end=stim_end_time)

        # run the analysis and print out a few of the features
        long_square_analysis = spa.LongSquareAnalysis(spfx, sptfx, subthresh_min_amp=-100.0)
        data = long_square_analysis.analyze(long_square_sweeps)    

        specimen_dict = summarize_ls_ephys_features(data)
        #data_dict[curr_file] = specimen_dict
        data_dict[row.file_name] = specimen_dict
        
    except (AssertionError, AttributeError, TypeError, FeatureError) as e:
    #except FeatureError as e:
        print(f'Error: {e} \n') # {row.filename} \n')

19o23003.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
INFO:root:Assigned subthreshold minimum amplitude of -100.000000.
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus_protocol_analysis.py:219: RuntimeWarning: Mean of empty slice
  features["tau"]

19o23001.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
INFO:root:Assigned subthreshold minimum amplitude of -100.000000.
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus_protocol_analysis.py:219: RuntimeWarning: Mean of empty slice
  features["tau"]

19o23010.nwb


...

INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
INFO:root:Assigned subthreshold minimum amplitude of -100.000000.
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus_protocol_analysis.py:219: RuntimeWarning: Mean of empty slice
  features["tau"]

19o09057.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
INFO:root:Assigned subthreshold minimum amplitude of -100.000000.
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus_protocol_analysis.py:219: RuntimeWarning: Mean of empty slice
  features["tau"]

19o08003.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
INFO:root:Assigned subthreshold minimum amplitude of -100.000000.
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus_protocol_analysis.py:219: RuntimeWarning: Mean of empty slice
  features["tau"]

In [40]:
dt_df = pd.DataFrame(data_dict)
m2019_longsquare_df = dt_df.T

In [41]:
m2019_longsquare_df

,adapt,avg_rate,first_isi,isi_cv,latency,mean_isi,median_isi,stim_amp,threshold_v,peak_v,...,fast_trough_t,trough_t,rheo_first_isi,v_baseline,rheobase_i,fi_fit_slope,sag,vm_for_sag,input_resistance,tau
19o23003.nwb,-0.009206,5.000,0.06070,0.044233,0.04425,0.058657,0.060200,100.000000,-46.539307,15.289307,...,0.25365,0.27115,0.20835,-61.022917,50.000000,0.039137,-88.609909,-64.178467,31.225433,NaN
19o23001.nwb,0.005959,5.625,0.05170,0.035529,0.01425,0.055506,0.055725,100.000000,-44.036865,28.839111,...,0.39140,0.41365,0.27560,-59.954530,50.000000,0.046429,-543.220581,-62.042240,11.712646,NaN
19o23010.nwb,0.012666,5.000,0.06150,0.059631,0.02505,0.064914,0.063450,250.000000,-38.116455,18.524170,...,0.29380,0.30440,0.13570,-68.056437,200.000000,0.048750,0.672731,-73.089600,4.293823,NaN
19o23039.nwb,0.009726,5.000,0.05935,0.044645,0.02815,0.063600,0.064000,150.000000,-47.760010,32.989502,...,0.33165,0.35530,0.14330,-64.021535,100.000000,0.026339,2.979283,-71.838379,13.820343,NaN
19o09016.nwb,0.001114,3.750,0.00350,1.153013,0.08365,0.061240,0.003850,200.000000,-33.752441,4.211426,...,0.23195,0.75900,NaN,-64.152048,150.000000,0.022768,0.899013,-74.249268,2.768250,NaN
2019_10_24_0037.nwb,0.021853,9.375,0.02400,0.175272,0.00450,0.032175,0.030600,150.000000,-37.567135,13.641357,...,0.22795,0.22805,0.04750,-60.183443,100.000000,0.041518,0.900426,-62.988281,21.267395,NaN
19o08018.nwb,0.008246,4.375,0.06570,0.043499,0.01985,0.070875,0.072150,200.000000,-37.475586,32.531738,...,0.26450,0.29345,0.15150,-62.076332,150.000000,0.025000,0.995954,-68.756104,6.431580,NaN
19o08027.nwb,0.018651,6.875,0.03555,0.116956,0.01400,0.043685,0.042700,250.000000,-35.095215,31.768799,...,0.25075,0.25395,0.04430,-62.921515,200.000000,0.017500,0.384710,-72.265625,5.557861,NaN
19o22018.nwb,0.006220,8.750,0.03410,0.064632,0.00045,0.037323,0.037250,150.000000,-36.224365,32.867432,...,0.22145,0.24770,0.14515,-67.150398,100.000000,0.053929,0.376820,-73.272705,2.698364,NaN
19o22031.nwb,0.011450,8.750,0.02970,0.075685,0.00735,0.034535,0.034700,150.000000,-38.269043,19.622803,...,0.24645,0.25075,0.04720,-68.348248,100.000000,0.040179,63.080006,-66.986084,52.917938,NaN


In [42]:
m2019_longsquare_df.to_csv(features_output_path / 'mouse2019_longsquares.csv')

## Process 2018 mouse longsquares

In [43]:
m2018meta.stimulus_type.value_counts()

longsquare    5
ramp          4
Name: stimulus_type, dtype: int64

In [44]:
m2018meta

,Cell Name,Data Type,Exp. Date,Cell #,Cell Layer,Stim Type,ZD status,stimulus_type,Gain,Offset,Response Channel,Command Channel,RMP
103,18130014,Mouse,Jan 30_2018,C5,L5,Firing,n.a,longsquare,1.0,15.2,Iclamp (mV),Current_in(pA),-75.2
5,2018_02_08_0000,Mouse,Feb 08_2018,C1,L5,Firing pattern,n.a,ramp,1.0,12.0,Iclamp (mV),Current_in(pA),-62.6
6,2018_02_08_0001,Mouse,Feb 08_2018,C1,L5,Firing pattern,n.a,longsquare,1.0,12.0,Iclamp (mV),Current_in(pA),-62.6
9,18208013,Mouse,Feb 08_2018,C3,L5,Firing pattern,n.a,ramp,1.0,27.3,Iclamp (mV),Current_in(pA),-62.3
10,18208014,Mouse,Feb 08_2018,C3,L5,Firing pattern,n.a,longsquare,1.0,27.3,Iclamp (mV),Current_in(pA),-62.3
14,18208023,Mouse,Feb 08_2018,C6,L5,Firing pattern,n.a,ramp,1.0,23.2,Iclamp (mV),Current_in(pA),-63.2
15,18208024,Mouse,Feb 08_2018,C6,L5,Firing pattern,n.a,longsquare,1.0,23.2,Iclamp (mV),Current_in(pA),-63.2
46,18118024,Mouse,Jan 18_2018,C4,L5,Firing pattern,n.a,longsquare,20.0,15.4,Iclamp (mV),Current_in(pA),-59.4
18,18208031,Mouse,Feb 08_2018,C8,L5,Firing pattern,n.a,ramp,1.0,22.4,Iclamp (mV),Current_in(pA),-62.4


In [45]:
cols_to_drop_m2018 = ['Cell Name', 'Data Type', 'Exp. Date', 'Stim Type', 'ZD status', 'Response Channel', 'Command Channel']
m2018 = m2018.merge(m2018meta, left_on='file_id', right_on='Cell Name').drop(cols_to_drop_m2018, axis=1)

In [46]:
data_dict = {}
for i, row in m2018[m2018.stimulus_type == 'longsquare'].iterrows():
    print(f'{row.file_name}')
    try:
        data_set = create_ephys_data_set(nwb_file=row.path, ontology=ontology_path)
        cell_features, sweep_features, cell_record, sweep_records, _, _ = extract_data_set_features(data_set)
        long_square_table = data_set.filtered_sweep_table(stimuli=data_set.ontology.long_square_names)
        long_square_sweeps = data_set.sweep_set(long_square_table.sweep_number)
        long_square_sweeps = data_set.sweep_set()

        # Select epoch corresponding to the actual recording from the sweeps
        # and align sweeps so that the experiment would start at the same time
        long_square_sweeps.select_epoch("recording")

        # find the start and end time of the stimulus
        # (treating the first sweep as representative)
        stim_start_index, stim_end_index = get_stim_epoch(long_square_sweeps.i[0], test_pulse=False)
        stim_start_time = long_square_sweeps.t[0][stim_start_index]
        stim_end_time = long_square_sweeps.t[0][stim_end_index]

        stim_start_time = 0.2
        stim_end_time = 1.8

        # build the extractors
        spfx = SpikeFeatureExtractor(start=stim_start_time, end=stim_end_time, filter = 1)
        sptfx = SpikeTrainFeatureExtractor(start=stim_start_time, end=stim_end_time)

        # run the analysis and print out a few of the features
        long_square_analysis = spa.LongSquareAnalysis(spfx, sptfx, subthresh_min_amp=-100.0)
        data = long_square_analysis.analyze(long_square_sweeps)    

        specimen_dict = summarize_ls_ephys_features(data)
        #data_dict[curr_file] = specimen_dict
        data_dict[row.file_name] = specimen_dict
        
    except (AssertionError, AttributeError, TypeError, FeatureError) as e:
    #except FeatureError as e:
        print(f'Error: {e} \n') # {row.filename} \n')

18118024.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephys

18208014.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephys

18208024.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephys

2018_02_08_0001.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephys

18130014.nwb


INFO:root:  
INFO:root:***** ***** ***** Analyzing cell features: ***** ***** *****
INFO:root:------------------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Long Squares: ***** ***** *****
INFO:root:-------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/ipfx/stimulus.py:88: UserWarning: Could not find stimulus: NA
  warnings.warn("Could not find stimulus: %s" % tag)
INFO:root:  
INFO:root:***** ***** ***** Short Squares: ***** ***** *****
INFO:root:--------------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Ramps: ***** ***** *****
INFO:root:------------------------------------------
INFO:root:  
INFO:root:  
INFO:root:***** ***** ***** Analyzing sweep features: ***** ***** *****
INFO:root:-------------------------------------------------------------
INFO:root:  
/Users/derek_howard/miniconda3/envs/nwbephys

In [47]:
dt_df = pd.DataFrame(data_dict)
m2018_longsquare_df = dt_df.T

In [48]:
m2018_longsquare_df

,adapt,avg_rate,first_isi,isi_cv,latency,mean_isi,median_isi,stim_amp,threshold_v,peak_v,...,fast_trough_t,trough_t,rheo_first_isi,v_baseline,rheobase_i,fi_fit_slope,sag,vm_for_sag,input_resistance,tau
18118024.nwb,0.007434,15.625,0.0191,0.106245,0.0202,0.021948,0.021975,200.000000,-8035.419434,17141.582031,...,0.2346,0.2347,0.0235,-24182.154236,150.000000,0.003750,0.036524,-9500.264648,6894.609375,NaN
18208014.nwb,0.012599,6.250,0.0711,0.079442,0.0308,0.082222,0.085100,59.999996,-36.760448,46.552544,...,0.5232,0.6181,0.4821,-58.181404,20.000000,0.027487,-158.124542,-90.166222,434.875518,NaN
18208024.nwb,NaN,1.250,0.1587,0.000000,0.0731,0.158700,0.158700,100.000000,-27.777637,43.328323,...,0.3049,0.3425,0.2451,-66.038666,59.999996,-0.010937,-480.091888,-106.818169,450.134307,NaN
2018_02_08_0001.nwb,-0.014184,5.000,0.1155,0.061153,0.0105,0.105714,0.103700,100.000000,-33.362309,57.274902,...,0.3037,0.3738,0.2469,-53.713541,59.999996,0.023065,-30.285553,-102.637207,320.434600,NaN
18130014.nwb,0.028985,5.000,0.0891,0.110098,0.0194,0.110286,0.110600,110.000000,-35.646782,40.647167,...,0.3222,0.3629,0.1712,-54.383799,70.000000,0.046875,86.506866,-98.207817,465.393066,NaN


In [49]:
m2018_longsquare_df.to_csv(features_output_path / 'mouse2018_longsquares.csv')